In [1]:
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.special import binom, beta, gamma
from scipy.optimize import minimize
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
import pandas as pd
sac = pd.read_csv('datasets/Sacramentorealestatetransactions.csv')
for x in sac.columns:
    if x != 'street':
        print sac[x].unique()

['SACRAMENTO' 'RANCHO CORDOVA' 'RIO LINDA' 'CITRUS HEIGHTS'
 'NORTH HIGHLANDS' 'ANTELOPE' 'ELK GROVE' 'ELVERTA' 'GALT' 'CARMICHAEL'
 'ORANGEVALE' 'FOLSOM' 'MATHER' 'POLLOCK PINES' 'GOLD RIVER'
 'EL DORADO HILLS' 'RANCHO MURIETA' 'WILTON' 'GREENWOOD' 'FAIR OAKS'
 'CAMERON PARK' 'LINCOLN' 'PLACERVILLE' 'MEADOW VISTA' 'ROSEVILLE'
 'ROCKLIN' 'AUBURN' 'LOOMIS' 'EL DORADO' 'PENRYN' 'GRANITE BAY'
 'FORESTHILL' 'DIAMOND SPRINGS' 'SHINGLE SPRINGS' 'COOL' 'WALNUT GROVE'
 'GARDEN VALLEY' 'SLOUGHHOUSE' 'WEST SACRAMENTO']
[95838 95823 95815 95824 95841 95842 95820 95670 95673 95822 95621 95833
 95660 95834 95843 95832 95827 95828 95758 95626 95825 95835 95624 95632
 95826 95608 95662 95630 95829 95655 95757 95742 95726 95818 95762 95831
 95683 95693 95814 95635 95864 95817 95628 95682 95648 95667 95610 95722
 95816 95678 95747 95819 95811 95677 95765 95603 95661 95650 95821 95623
 95663 95746 95631 95619 95614 95690 95633 95691]
['CA']
[2 3 1 4 0 5 8 6]
[1 2 3 4 0 5]
[ 836 1167  796  852  797 1122 

In [3]:
print sac.columns

Index([u'street', u'city', u'zip', u'state', u'beds', u'baths', u'sq__ft',
       u'type', u'sale_date', u'price', u'latitude', u'longitude'],
      dtype='object')


In [4]:
sac = sac[(sac.beds != 0) & (sac.baths != 0) & (sac['sq__ft'] != 0) & (sac['type'] != 'Unknown')]

In [5]:
sac['$/ft'] = sac['price'] / sac['sq__ft']

In [6]:
sac = sac[sac['$/ft'] >= 1]
sac = sac.loc[:, \
            ['zip','$/ft','beds','baths','type']]


In [7]:
nc = [x for x in sac.columns if x != 'type' and x != 'zip']

sac.ix[:, nc] = (sac.ix[:, nc] - sac.ix[:, nc].min())\
                   / (sac.ix[:, nc].max() - sac.ix[:, nc].min())

In [8]:
pd.get_dummies(sac['zip'])
sac = pd.concat([sac, pd.get_dummies(sac['zip'])], axis=1)
sac.reset_index(inplace=True)
sac = sac.drop('index',axis=1)
sac.head()

,zip,$/ft,beds,baths,type,95603,95608,95610,95614,95619,...,95831,95832,95833,95834,95835,95838,95841,95842,95843,95864
0,95838,0.075952,0.142857,0.0,Residential,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,95823,0.055093,0.285714,0.0,Residential,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,95815,0.102374,0.142857,0.0,Residential,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,95815,0.093642,0.142857,0.0,Residential,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,95824,0.129696,0.142857,0.0,Residential,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
from sklearn import linear_model as lm

In [20]:
xl = sac.iloc[:,5:].columns
mylist = list(xl)

In [ ]:
myx = ['beds','baths']
for x in myx:
    mylist.append(x)
x = sac[mylist]
y = sac['$/ft']
x

In [27]:
housecost = lm.RidgeCV(alphas=[.1,1,10,100],cv=5)
housecost.fit(x,y)

RidgeCV(alphas=[0.1, 1, 10, 100], cv=5, fit_intercept=True, gcv_mode=None,
    normalize=False, scoring=None, store_cv_values=False)

In [28]:
housecost.coef_

array([  5.46770960e-02,   6.49582533e-03,  -4.51275493e-02,
         1.93664255e-02,  -1.35695141e-02,  -6.34035068e-02,
         7.98486072e-04,  -6.43731839e-02,  -9.57492705e-02,
        -4.61172649e-03,   2.99058838e-02,  -5.45164826e-02,
        -3.50106391e-02,  -1.38492098e-02,   2.25402810e-01,
        -5.41845848e-02,  -9.37951702e-02,   1.31181795e-02,
        -2.84306190e-03,   1.74119927e-01,   8.82277297e-02,
        -4.45952825e-02,  -3.60126337e-02,  -2.55976077e-03,
        -2.53182996e-02,   1.95079581e-02,  -3.50962897e-02,
         3.72758904e-02,  -1.46364424e-02,   2.19635527e-02,
         1.45017340e-02,   1.05088591e-02,  -5.97805380e-02,
         1.01085719e-01,   7.05698781e-03,  -2.20729075e-02,
        -2.98178598e-02,   8.75476235e-03,   4.18169443e-02,
         1.38631354e-01,   6.25744299e-02,  -8.66754123e-02,
         1.16668338e-01,  -4.96557631e-02,   1.03995548e-01,
         1.63857083e-01,  -1.17278527e-02,  -5.38321119e-03,
        -3.74096522e-02,

In [31]:
housecost.score(x,y)
#housecost.predict(x.iloc[0,:])

0.36510149557796578

In [33]:
housecost.predict(x.iloc[0,:])

//anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([ 0.1622737])

**Exercise:**

1. Create a linear regression model using scikit - learn (ridge regression for example) that predicts the housing price (Maybe ST, WAY or DR has an impact in the value?). Discuss the features you are using. Which ones do you need to categorize? 
2. Use Cross Validation to find the best regularization constant.
3. Look up sklearn.preprocessing.PolynomialFeatures and use the method to generate ridge regression models with various polynomial degrees. Use cross validation to find the best degree.

In [34]:
from sklearn import preprocessing as pp
house2 = pp.PolynomialFeatures()

In [35]:
house2.fit(x,y)

PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)

In [39]:
house2.transform(x,y)

array([[ 1.        ,  0.        ,  0.        , ...,  0.02040816,
         0.        ,  0.        ],
       [ 1.        ,  0.        ,  0.        , ...,  0.08163265,
         0.        ,  0.        ],
       [ 1.        ,  0.        ,  0.        , ...,  0.02040816,
         0.        ,  0.        ],
       ..., 
       [ 1.        ,  0.        ,  0.        , ...,  0.08163265,
         0.07142857,  0.0625    ],
       [ 1.        ,  0.        ,  0.        , ...,  0.18367347,
         0.10714286,  0.0625    ],
       [ 1.        ,  0.        ,  0.        , ...,  0.08163265,
         0.07142857,  0.0625    ]])

In [41]:
house2.fit_transform(x,y)

array([[ 1.        ,  0.        ,  0.        , ...,  0.02040816,
         0.        ,  0.        ],
       [ 1.        ,  0.        ,  0.        , ...,  0.08163265,
         0.        ,  0.        ],
       [ 1.        ,  0.        ,  0.        , ...,  0.02040816,
         0.        ,  0.        ],
       ..., 
       [ 1.        ,  0.        ,  0.        , ...,  0.08163265,
         0.07142857,  0.0625    ],
       [ 1.        ,  0.        ,  0.        , ...,  0.18367347,
         0.10714286,  0.0625    ],
       [ 1.        ,  0.        ,  0.        , ...,  0.08163265,
         0.07142857,  0.0625    ]])